In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import itertools
from scipy.spatial import distance
# %matplotlib inline

In [2]:
def unit_vector(A,B):
    # Calculate the unit vector from A to B 

    dist = distance.euclidean(A,B)

    return ((B[0]-A[0])/dist,(B[1]-A[1])/dist)

def calc_force(l, myosin,b):

    return 10*(l-1) + b*myosin

def d_pos(position,force,dt):

    x_new = position[0] + (dt/const.eta)*force[0]

    y_new = position[1] + (dt/const.eta)*force[1]

    return (x_new,y_new)

def CellArea(corners):
    n = len(corners) # of corners
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area) / 2.0
    return area


In [3]:
def tissue():
    def gen_nodes(ori):
        nodes = []
        for n in range(0,6):
            nodes.append((ori[0] + r*np.cos(n*np.pi/3), ori[1] + r*np.sin(n*np.pi/3)))
        return nodes

    def add_nodes(nodes, i):
        pos = nx.get_node_attributes(G,'pos')
        cen_index = i-1
        centers.append(cen_index)
        boundary = []
        spokes = []
        for node in nodes:
            add_node = True
            for existing_node in range(0,len(pos)):
                if distance.euclidean(pos[existing_node],node) < 10**(-7):
                    add_node = False
                    boundary.append(existing_node)
                    spokes.append((cen_index,existing_node))
                    break

            if add_node == True:
                G.add_node(i,pos=node)
                i += 1
                boundary.append(i-1)
                spokes.append((cen_index,i-1))

        return boundary, spokes, i

    def add_spokes_edges(spokes, boundary):
        boundary.append(boundary[0])
        G.add_edges_from(spokes,beta=10,myosin=1000)    
        G.add_path(boundary,beta=0,myosin=0,color='r')

        return

    G = nx.Graph()

    r = 7.6        # initial spoke length
    num_cells = 5  # number of cells in center row

    centers = []
    i = 0
    # Center cell set up
    origin = (0.0,0.0)
    G.add_node(i,pos=origin)
    i += 1

    nodes = gen_nodes(origin)
    boundary, spokes, i = add_nodes(nodes,i)
    add_spokes_edges(spokes, boundary)

    for index in range(1,int((num_cells - 1)/2.)+1):
        # # Step Up
        origin = (0, np.sqrt(3)*r*index)
        G.add_node(i,pos=origin)
        i += 1

        nodes = gen_nodes(origin)
        boundary, spokes, i = add_nodes(nodes,i)
        add_spokes_edges(spokes, boundary)

        # # # Step down
        origin = (0, -np.sqrt(3)*r*index)
        G.add_node(i,pos=origin)
        i += 1

        nodes = gen_nodes(origin)
        boundary, spokes, i = add_nodes(nodes,i)
        add_spokes_edges(spokes, boundary)

    for index in range(1,num_cells):  
        if (num_cells - index) % 2 == 0:
            for j in range(1,(num_cells-index),2):
                origin = ((3/2.)*r*index,(np.sqrt(3)/2.)*r*j)
                G.add_node(i,pos=origin)
                i += 1

                nodes = gen_nodes(origin)
                boundary, spokes, i = add_nodes(nodes,i)
                add_spokes_edges(spokes, boundary)

                origin = ((3/2.)*r*index,(-np.sqrt(3)/2.)*r*j)
                G.add_node(i,pos=origin)
                i += 1

                nodes = gen_nodes(origin)
                boundary, spokes, i = add_nodes(nodes,i)
                add_spokes_edges(spokes, boundary)

            # Step Left

                origin = (-(3/2.)*r*index,(np.sqrt(3)/2.)*r*j)
                G.add_node(i,pos=origin)
                i += 1

                nodes = gen_nodes(origin)
                boundary, spokes, i = add_nodes(nodes,i)
                add_spokes_edges(spokes, boundary)

                origin = (-(3/2.)*r*index,(-np.sqrt(3)/2.)*r*j)
                G.add_node(i,pos=origin)
                i += 1

                nodes = gen_nodes(origin)
                boundary, spokes, i = add_nodes(nodes,i)
                add_spokes_edges(spokes, boundary)

        else:
            for j in range(0,(num_cells-index),2):
                origin = (3*(1/2.)*r*index, (np.sqrt(3)/2.)*r*j)
                G.add_node(i,pos=origin)
                i += 1

                nodes = gen_nodes(origin)
                boundary, spokes, i = add_nodes(nodes,i)
                add_spokes_edges(spokes, boundary)
                
                if j != 0:
                    origin = (3*(1/2.)*r*index, -(np.sqrt(3)/2.)*r*j)
                    G.add_node(i,pos=origin)
                    i += 1

                    nodes = gen_nodes(origin)
                    boundary, spokes, i = add_nodes(nodes,i)
                    add_spokes_edges(spokes, boundary)

                # Step Left
                origin = (-3*(1/2.)*r*index, (np.sqrt(3)/2.)*r*j)
                G.add_node(i,pos=origin)
                i += 1

                nodes = gen_nodes(origin)
                boundary, spokes, i = add_nodes(nodes,i)
                add_spokes_edges(spokes, boundary)
                
                if j != 0:
                    origin = (-3*(1/2.)*r*index, -(np.sqrt(3)/2.)*r*j)
                    G.add_node(i,pos=origin)
                    i += 1

                    nodes = gen_nodes(origin)
                    boundary, spokes, i = add_nodes(nodes,i)
                    add_spokes_edges(spokes, boundary)

    nx.set_node_attributes(G, 'time_lag', 0)
    
    for j in centers:
            G.node[j]['time_lag'] = np.random.randint(0,2000)
    
    boundary = []
    for j in G.nodes_iter():
        if G.degree(j) == 3 or G.degree(j) == 5:
            boundary.append(j)
    
    return G, centers, boundary

In [4]:
G, centers, boundary = tissue()
pos = nx.get_node_attributes(G,'pos')
nx.draw(G,pos, with_labels=True,node_size=2)
plt.axis("on")
plt.grid("on")
plt.axis('equal')
plt.show()

In [5]:
pos = nx.get_node_attributes(G,'pos')
edges,colors = zip(*nx.get_edge_attributes(G,'color').items())
nx.draw(G,pos, node_size = 1, edgelist=edges,edge_color=colors,width=1)
plt.axis("on")
plt.grid("on")
plt.axis('equal')
plt.show()

In [6]:
centers

[0,
 7,
 12,
 17,
 22,
 27,
 31,
 34,
 38,
 41,
 44,
 47,
 50,
 53,
 57,
 61,
 64,
 67,
 70,
 73,
 77,
 80,
 84,
 87,
 91]

In [7]:
G.degree(0)

6

In [8]:
boundary = []
for j in G.nodes_iter():
    if G.degree(j) == 3:
        boundary.append(j)
boundary 

[19,
 20,
 25,
 26,
 43,
 46,
 48,
 52,
 63,
 66,
 68,
 72,
 75,
 79,
 81,
 86,
 88,
 89,
 90,
 92,
 93,
 94]